In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            print("Name:", gpu.name, "  Type:", gpu.device_type)

    except RuntimeError as e:
        print(e)

Name: /physical_device:GPU:0   Type: GPU


In [2]:
import pandas as pd
import numpy as np
import math 
import matplotlib.pyplot as plt
from sklearn.utils import shuffle as skshuffle
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Input
from sklearn import preprocessing
from tensorflow.keras import backend as K
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
import pyswarms as ps
import random
from pylab import rcParams
%matplotlib inline

In [3]:
loss_tracker = keras.metrics.Mean(name="loss")
mae_metric = keras.metrics.MeanAbsoluteError(name="mae")

In [4]:
rcParams['figure.figsize'] = 50, 10

In [5]:
PATH_TO_TEST_TRAIN_DATA = 'Tangni/Data/'

In [6]:
class MLP_Tangni:
    def __init__(self,dictionary):
        
        self.train_size = 62
        
        
        self.data1 = np.array(pd.read_csv(PATH_TO_TEST_TRAIN_DATA + '1-3.csv',  header=0, usecols=[0, 1], index_col=0))
        self.data2 = np.array(pd.read_csv(PATH_TO_TEST_TRAIN_DATA + '2-12.csv', header=0, usecols=[0, 1], index_col=0))
        self.data3 = np.array(pd.read_csv(PATH_TO_TEST_TRAIN_DATA + '3-6.csv',  header=0, usecols=[0, 1], index_col=0))
        self.data4 = np.array(pd.read_csv(PATH_TO_TEST_TRAIN_DATA + '4-15.csv', header=0, usecols=[0, 1], index_col=0))
        self.data5 = np.array(pd.read_csv(PATH_TO_TEST_TRAIN_DATA + '5-15.csv', header=0, usecols=[0, 1], index_col=0))     
        
    def Print(self):
        print(self.data1)
        
    def rmse(self, true, predict):
        
        r = true - predict
        sq = r*r
        avg = np.mean(sq)
        error = math.sqrt(avg)

        return error
    
    def Average(self, lst):
        return sum(lst) / len(lst)
    
    def dataProcessing(self, dF, lookback):
    
        data=list()
        label=list()
        for i in range(len(dF) - lookback):

            data.append(np.array(dF[i:i+lookback]))

            label.append(np.array(dF[i+lookback]))
        
        return np.array(data), np.array(label)

    def make_packets(self, lookback):
        
        X1, L1 = self.dataProcessing(self.data1, lookback)
        X2, L2 = self.dataProcessing(self.data2, lookback)
        X3, L3 = self.dataProcessing(self.data3, lookback)
        X4, L4 = self.dataProcessing(self.data4, lookback)
        X5, L5 = self.dataProcessing(self.data5, lookback)
        
        train_X1, label_X1,test_X1,tst_label_X1 = X1[:self.train_size - lookback], L1[:self.train_size - lookback], X1[self.train_size - lookback:], L1[self.train_size - lookback:]
        train_X2, label_X2,test_X2,tst_label_X2 = X2[:self.train_size - lookback], L2[:self.train_size - lookback], X2[self.train_size - lookback:], L2[self.train_size - lookback:]
        train_X3, label_X3,test_X3,tst_label_X3 = X3[:self.train_size - lookback], L3[:self.train_size - lookback], X3[self.train_size - lookback:], L3[self.train_size - lookback:]
        train_X4, label_X4,test_X4,tst_label_X4 = X4[:self.train_size - lookback], L4[:self.train_size - lookback], X4[self.train_size - lookback:], L4[self.train_size - lookback:]
        train_X5, label_X5,test_X5,tst_label_X5 = X5[:self.train_size - lookback], L5[:self.train_size - lookback], X5[self.train_size - lookback:], L5[self.train_size - lookback:]
        
    
        return train_X1, label_X1,test_X1,tst_label_X1, train_X2, label_X2,test_X2,tst_label_X2,train_X3, label_X3,test_X3,tst_label_X3, train_X4, label_X4,test_X4,tst_label_X4, train_X5, label_X5,test_X5,tst_label_X5

    def getData(self, lookback = 1, shuffle=0):
        
        train_X1, label_X1,test_X1,tst_label_X1, train_X2, label_X2,test_X2,tst_label_X2,train_X3, label_X3,test_X3,tst_label_X3, train_X4, label_X4,test_X4,tst_label_X4, train_X5, label_X5,test_X5,tst_label_X5 = self.make_packets(lookback)
        DATA = [train_X1, label_X1,test_X1,tst_label_X1, train_X2, label_X2,test_X2,tst_label_X2,train_X3, label_X3,test_X3,tst_label_X3, train_X4, label_X4,test_X4,tst_label_X4, train_X5, label_X5,test_X5,tst_label_X5]
        
        Train = np.append(train_X1, np.append(train_X2, np.append(train_X3, np.append(train_X4, train_X5, axis=0), axis=0), axis=0), axis=0)
        Label = np.append(label_X1, np.append(label_X2, np.append(label_X3, np.append(label_X4, label_X5, axis=0), axis=0), axis=0), axis=0) 
     
        if (shuffle == 1):
            Train, Label = skshuffle(Train, Label, random_state=0)
            
        return Train, Label
    
    def MLP_model(self, units, lookback, features):
        
        model = Sequential()
        model.add(Dense(units = units, activation= 'tanh', input_shape = (lookback, features)))
        model.add(Dense(units = 1, activation= 'linear'))
        model.compile(optimizer = 'adam', loss = 'mean_squared_error')
#         model.summary()
        
        return model
    
    def trainModel(self, units, lookback = 1, batch_size = 4, shuffle = 0):
        
        train_X1, label_X1,test_X1,tst_label_X1, train_X2, label_X2,test_X2,tst_label_X2,train_X3, label_X3,test_X3,tst_label_X3, train_X4, label_X4,test_X4,tst_label_X4, train_X5, label_X5,test_X5,tst_label_X5 = self.make_packets(lookback)
        DATA = [train_X1, label_X1,test_X1,tst_label_X1, train_X2, label_X2,test_X2,tst_label_X2,train_X3, label_X3,test_X3,tst_label_X3, train_X4, label_X4,test_X4,tst_label_X4, train_X5, label_X5,test_X5,tst_label_X5]
        
        Train = np.append(train_X1, np.append(train_X2, np.append(train_X3, np.append(train_X4, train_X5, axis=0), axis=0), axis=0), axis=0)
        Label = np.append(label_X1, np.append(label_X2, np.append(label_X3, np.append(label_X4, label_X5, axis=0), axis=0), axis=0), axis=0) 
     
        if (shuffle == 1):
            Train, Label = skshuffle(Train, Label, random_state=0)
            
        LSTM = self.MLP_model(units, lookback)
        LSTM.fit(Train, Label, epochs = 3, batch_size = batch_size, verbose=0)
        
        RMSE = list()
        
        #Calculate the error of the traing and testing
        for i in range(0, 20, 2):
            
            prediction = LSTM.predict(DATA[i])
            RMSE.append(self.rmse(prediction, DATA[i+1]))
           
         
        #Find average of the training and testing
        RMSE.append(self.Average(RMSE[0:5]))
        RMSE.append(self.Average(RMSE[5:10]))
        
      
        return RMSE
    
    def accuracy(self, MLP, Data1, Data2, Data3, shape, r):
        
        P =[]
           
        if r==0:
            D1 = np.copy(Data1[0])
            D1=D1.reshape(shape)
            D2 = np.copy(Data2[1:])
            Testing = np.copy(Data3)
            for i in range(len(D2)):
                p=MLP.predict(D1)
                P.append(p[-1][0])
                a=np.array(D2[i])
                a=a.reshape(shape)
                D1=np.append(D1,a,axis=0)
            P=np.array(P)
            RMSE = math.sqrt(mean_squared_error(Testing[1:], P))
            MAE  = mean_absolute_error(Testing[1:], P)
            MAPE = mean_absolute_percentage_error(Testing[1:], P)
            
        if r==1:
            D1 = np.copy(Data1)
            D2 = np.copy(Data2)
            Testing = np.copy(Data3)
            for i in range(16):
                p=MLP.predict(D1)
                P.append(p[-1][0])
                a=np.array(D2[i])
                a=a.reshape(shape)
                D1=np.append(D1,a,axis=0)
            P=np.array(P)
            RMSE = math.sqrt(mean_squared_error(Testing, P))
            MAE  = mean_absolute_error(Testing, P)
            MAPE = mean_absolute_percentage_error(Testing, P)
            
        return RMSE, MAE, MAPE, P
    
    def runModel(self, units, features, lookback = 1, epochs=1, batch_size = 4, shuffle = 0):
        
        train_X1, label_X1,test_X1,tst_label_X1, train_X2, label_X2,test_X2,tst_label_X2,train_X3, label_X3,test_X3,tst_label_X3, train_X4, label_X4,test_X4,tst_label_X4, train_X5, label_X5,test_X5,tst_label_X5 = self.make_packets(lookback)
        DATA = [train_X1, label_X1,test_X1,tst_label_X1, train_X2, label_X2,test_X2,tst_label_X2,train_X3, label_X3,test_X3,tst_label_X3, train_X4, label_X4,test_X4,tst_label_X4, train_X5, label_X5,test_X5,tst_label_X5]
        
        Train = np.append(train_X1, np.append(train_X2, np.append(train_X3, np.append(train_X4, train_X5, axis=0), axis=0), axis=0), axis=0)
        Label = np.append(label_X1, np.append(label_X2, np.append(label_X3, np.append(label_X4, label_X5, axis=0), axis=0), axis=0), axis=0) 
     
        if (shuffle == 1):
            Train, Label = skshuffle(Train, Label, random_state=0)
        
        MLP = self.MLP_model(units, lookback,features)
        
        MLP.fit(Train, Label, epochs = epochs, batch_size = batch_size, verbose=0)
        
        RMSE = []
        MAE = []
        MAPE = []
        P =[]
        Pre=[]
        T =[]
        sh = (1,lookback,features)
        
        #Calculate the error of the traing and testing
        
        A, B, C, D = self.accuracy(MLP,train_X1,train_X1,label_X1,sh,r=0)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(MLP,train_X2,train_X2,label_X2,sh,r=0)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(MLP,train_X3,train_X3,label_X3,sh,r=0)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(MLP,train_X4,train_X4,label_X4,sh,r=0)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(MLP,train_X5,train_X5,label_X5,sh,r=0)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(MLP,train_X1,test_X1,tst_label_X1,sh,r=1)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(MLP,train_X2,test_X2,tst_label_X2,sh,r=1)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(MLP,train_X3,test_X3,tst_label_X3,sh,r=1)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(MLP,train_X4,test_X4,tst_label_X4,sh,r=1)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(MLP,train_X5,test_X5,tst_label_X5,sh,r=1)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        RMSE.append(np.sum(RMSE[:5])/5)
        RMSE.append(np.sum(RMSE[5:])/5)
        
        
        return RMSE, MAE, MAPE, Pre
    
    def runModel_weights(self, units, features, weights, lookback = 1, epochs=1, batch_size = 4, shuffle = 0):
        
        train_X1, label_X1,test_X1,tst_label_X1, train_X2, label_X2,test_X2,tst_label_X2,train_X3, label_X3,test_X3,tst_label_X3, train_X4, label_X4,test_X4,tst_label_X4, train_X5, label_X5,test_X5,tst_label_X5 = self.make_packets(lookback)
        DATA = [train_X1, label_X1,test_X1,tst_label_X1, train_X2, label_X2,test_X2,tst_label_X2,train_X3, label_X3,test_X3,tst_label_X3, train_X4, label_X4,test_X4,tst_label_X4, train_X5, label_X5,test_X5,tst_label_X5]
        
        Train = np.append(train_X1, np.append(train_X2, np.append(train_X3, np.append(train_X4, train_X5, axis=0), axis=0), axis=0), axis=0)
        Label = np.append(label_X1, np.append(label_X2, np.append(label_X3, np.append(label_X4, label_X5, axis=0), axis=0), axis=0), axis=0) 
     
        if (shuffle == 1):
            Train, Label = skshuffle(Train, Label, random_state=0)
        
        MLP = self.MLP_model(units, lookback, features)
        
        idx1 = features * units
        idx2 = idx1 + units
        idx3 = idx2 + units
        idx4 = idx3 + 1


        W1 = weights[0:idx1].reshape((features, units))

        W2 = weights[idx1:idx2].reshape((units))

        W3 = weights[idx2:idx3].reshape((units,1))

        W4 = weights[idx3:].reshape((1))


        W = [W1,W2,W3,W4]


        MLP.set_weights(W)
        
        MLP.fit(Train, Label, epochs = epochs, batch_size = batch_size, verbose=0)
        
        RMSE = []
        MAE = []
        MAPE = []
        P =[]
        Pre=[]
        T =[]
        sh = (1,lookback,features)
        
        #Calculate the error of the traing and testing
        
        A, B, C, D = self.accuracy(MLP,train_X1,train_X1,label_X1,sh,r=0)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(MLP,train_X2,train_X2,label_X2,sh,r=0)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(MLP,train_X3,train_X3,label_X3,sh,r=0)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(MLP,train_X4,train_X4,label_X4,sh,r=0)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(MLP,train_X5,train_X5,label_X5,sh,r=0)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(MLP,train_X1,test_X1,tst_label_X1,sh,r=1)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(MLP,train_X2,test_X2,tst_label_X2,sh,r=1)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(MLP,train_X3,test_X3,tst_label_X3,sh,r=1)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(MLP,train_X4,test_X4,tst_label_X4,sh,r=1)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        A, B, C, D = self.accuracy(MLP,train_X5,test_X5,tst_label_X5,sh,r=1)
        RMSE.append(A),MAE.append(B),MAPE.append(C),Pre.append(D)
        
        RMSE.append(np.sum(RMSE[:5])/5)
        RMSE.append(np.sum(RMSE[5:])/5)
        
        
        return RMSE, MAE, MAPE, Pre
    
            

In [7]:
dictionary = {'normalize': 0}

In [8]:
mlp = MLP_Tangni(dictionary)

In [9]:
class CustomModel(keras.Model):
    def train_step(self, data):

        x, y = data
        
        y_pred = self(x, training=True)
        loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)

         # Compute our own metrics
        loss_tracker.update_state(loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

    # Forward propagation
def forward_props(params, units, features):
    
    idx1 = features * units
    idx2 = idx1 + units
    idx3 = idx2 + units
    idx4 = idx3 + 1
    
    
    W1 = params[0:idx1].reshape((features, units))
   
    W2 = params[idx1:idx2].reshape((units))
    
    W3 = params[idx2:idx3].reshape((units,1))
    
    W4 = params[idx3:].reshape((1))
    

    W = [W1,W2,W3,W4]
   
    modelOne.set_weights(W)
    
    history = modelOne.fit(Train, Label, epochs = 1, batch_size = 1000, verbose=0)
    
    
    loss = history.history['loss'][0]

    return loss

def fun(x,print_step):

    n_particles = x.shape[0]
    
    j = [forward_props(x[i],units, features) for i in range(n_particles)]
    return np.array(j)

In [10]:
lookback = [1,2,3,4,5]
unitss = [10, 50, 100, 200, 300, 400, 500]
Parameters = []
Performance =[]
minimum = 100
idx = 0
minidx = 0

for i in lookback:
        for u in unitss:

            Parameters.append([u,i])

            units = u
            features = 1
            
            Train, Label =mlp.getData(lookback=i, shuffle=0)

            inputs = keras.Input(shape=(i,features))
            L = keras.layers.Dense(units = units, activation='tanh')(inputs)
            outputs = keras.layers.Dense(1, activation='linear')(L)
            modelOne = CustomModel(inputs, outputs)
            modelOne.compile(optimizer="adam", loss="mse", metrics=['mae'])

            # Initialize swarm
            options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

            # Call instance of PSO

            dimensions = features * units + units + units + 1
            optimizer = ps.single.GlobalBestPSO(n_particles=100, dimensions=dimensions, options=options)

            # Perform optimization
            cost, pos = optimizer.optimize(fun, print_step=0, iters=100, verbose=1)

            RMSE,MAE,MAPE,Pre = mlp.runModel_weights(units=u, features=1, weights=pos, lookback = i, epochs=1000, batch_size = 64, shuffle = 0)

            print([u,i],RMSE[-2],RMSE[-1])
            Performance.append([u,i,RMSE[-2],RMSE[-1]])

            if RMSE[-2] < minimum:
                minimum = RMSE[-2]
                minidx = idx

            idx += 1
                
print('--------------------------------------------------')
print('Minimimu Error at [units, lookback, shuffle]')

print(Performance[minidx])

2022-03-16 11:41:02,259 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|█████████████████████|100/100, best_cost=0.294
2022-03-16 11:46:28,040 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.2939084768295288
2022-03-16 11:46:49,632 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}


[10, 1] 0.5498050027921394 0.6111151201728487


pyswarms.single.global_best: 100%|█████████████████████|100/100, best_cost=0.269
2022-03-16 11:52:12,737 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.26872915029525757
2022-03-16 11:52:33,294 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}


[50, 1] 0.520927438104889 0.6132063968101755


pyswarms.single.global_best: 100%|█████████████████████|100/100, best_cost=0.293
2022-03-16 11:57:58,300 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.2933683693408966
2022-03-16 11:58:18,751 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}


...

[300, 5] 0.7880046973363182 0.8528646290481804


pyswarms.single.global_best: 100%|█████████████████████|100/100, best_cost=0.595
2022-03-16 14:44:40,169 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.5952896475791931
2022-03-16 14:44:58,873 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}


[400, 5] 0.795478141110786 0.8576668556521373


pyswarms.single.global_best: 100%|█████████████████████|100/100, best_cost=0.666
2022-03-16 14:50:03,873 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.666267991065979


[500, 5] 0.7920999031844256 0.8488437442956819
--------------------------------------------------
Minimimu Error at [units, lookback, shuffle]
[50, 1, 0.520927438104889, 0.6132063968101755]


In [13]:
units = 50
features = 1
lookback = 1
shuffle = 0

Train, Label =mlp.getData(lookback=lookback, shuffle=shuffle)
Train.shape

inputs = keras.Input(shape=(lookback,features))
L = keras.layers.Dense(units = units, activation='tanh')(inputs)
outputs = keras.layers.Dense(1, activation='linear')(L)
modelOne = CustomModel(inputs, outputs)
modelOne.compile(optimizer="adam", loss="mse", metrics=['mae'])

# Initialize swarm
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

# Call instance of PSO

dimensions = features * units + units + units + 1
optimizer = ps.single.GlobalBestPSO(n_particles=100, dimensions=dimensions, options=options)

# Perform optimization
cost, posfinal = optimizer.optimize(fun, print_step =0, iters=1000, verbose=1)

2022-03-16 15:53:01,481 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|███████████████████|1000/1000, best_cost=0.198
2022-03-16 16:42:53,944 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.19787602126598358


In [12]:
RMSE,MAE,MAPE,Pre = mlp.runModel_weights(units=50, features=1, weights=posfinal, lookback = 1, epochs=500, batch_size = 4, shuffle = 0)

In [18]:
RMSE

[1.4755823913165909,
 0.11416946348623518,
 0.2251136159461647,
 0.6599336221166637,
 1.4857004230564732,
 0.02122173064648436,
 0.061810158304199,
 1.4481377782802172,
 0.14067540898465505,
 1.7802737420784287,
 0.7920999031844256,
 0.8488437442956819]

In [17]:
print(RMSE[0],RMSE[1],RMSE[2],RMSE[3],RMSE[4],RMSE[5],RMSE[6],RMSE[7],RMSE[8],RMSE[9])
print(MAE[0],MAE[1],MAE[2],MAE[3],MAE[4],MAE[5],MAE[6],MAE[7],MAE[8],MAE[9])
print(MAPE[0],MAPE[1],MAPE[2],MAPE[3],MAPE[4],MAPE[5],MAPE[6],MAPE[7],MAPE[8],MAPE[9])


1.4755823913165909 0.11416946348623518 0.2251136159461647 0.6599336221166637 1.4857004230564732 0.02122173064648436 0.061810158304199 1.4481377782802172 0.14067540898465505 1.7802737420784287
0.7100726234391893 0.07345435353727156 0.21651581242769666 0.1987094670671245 1.0179955013402309 0.02122173064648436 0.061810158304199 0.8137173954094677 0.11232755687821197 1.3200325716193229
0.7561874092134665 0.02984837680163412 94.3546164273124 0.05974633149322047 1.7462619448302128 0.0038361569573248395 0.025116726325775184 9.850942888018006 0.033591676378751475 1.775397539339988
